# Lecture 4: Graph as Matrix

이 강의에서 배우는 것
- random walk를 통해 node importance 결정(PageRank)
- matrix factorization을 통한 node embedding
- Node2Vec와 같은 다른 node embedding들도 matrix factorization의 관점에서 볼 수 있음

random walk, matrix factorization, node embedding은 서로 연관이 있음.

Link Analysis approaches to compute the importance of nodes in a graph:
- PageRank
- Personlized PageRank(PPR)
- Random Walk with Restarts

## Lecture 4.1: PageRank

**The Flow Model**

주어진 graph의 node $j$의 importance $r_j$는 다음과 같이 나타낼 수 있다.
$$
r_j = \sum_{i \rightarrow j} \frac{r_i}{d_i}
$$
여기서, $i \rightarrow j$는 node $i$에서 node $j$로 in-link를 나타내고, $d_i$는 node $i$의 out-degree이다.

또한, $i \rightarrow j$일 때, $(j,i)$-성분이 $1/d_i$인 행렬 $M$을 생각하면, flow equation을 다음과 같이 쓸 수 있다. (이런 행렬 $M$을 stochastic adjacency matrix라고 한다.)
$$
r = Mr
$$
그리고 rank vector $r$은 principle eigenvector로 볼 수도 있고, 그래프 위에서 random walk의 stationary distribution으로도 볼 수 있다.

## Lecture 4.2: PageRank: How to solve?

**Power Iteration Method**

위의 flow equation은 다음과 같은 power iteration을 통해서 풀 수 있다.
- intialize $r^{(0)} = [1/N, \cdots, 1/N]^\top$
- iterate $r^{(t+1)} = Mr^{(t)}$
- stop when $|r^{(t+1)} - r^{(t)}|_1 < \epsilon$

그런데,
- 위의 방법이 실제로 수렴할까?
- 우리가 원하는 방향으로 수렴할까?
- reasonable한 결과가 나올까?

하는 의문이 남는다. 또한, 실제로 다음과 같은 경우에 문제가 발생하기도 한다.
- out-link가 없는 경우(dead end)
    - importance가 새 나감.
- out-link가 자기 자신만을 가리키는 경우(spider trap)
    - 결국 한쪽으로 모든 importance가 쏠림.

**Solution to Spider Traps**

매 time step마다 다음 두 가지 선택을 한다.
- follow a link at random with probability $\beta$.
- jump to a random page with probability $1-\beta$.

**Solution to Dead Ends**

dead end에서 임의의 node로 teleport한다.

**The Google Matrix**

The Google Matrix $G$:
$$
G = \beta M + \frac{(1-\beta)}{N} \mathbf{1} \mathbf{1}^\top
$$
, $[1/N]_{N \times N}$ is an $N$ by $N$ matrix whose entries are all $1/N$. 

(그런데, 이 matrix는 dead end에 대한 solution은 들어가있지 않은 것 아닌지..?)

## Lecture 4.3: Random Walk with Restarts and Personalized PageRank

bipartite graph를 생각하자. (예를 들어, user와 item간의 interaction을 나타내는 graph) 이때, 어떤 item Q와 가장 연관이 있는 item은 무엇인지 어떻게 찾을 수 있을까?

- Personalized PageRank: 기본적인 PageRank와는 다르게 전체 node의 부분집합으로만 teleport한다.
- Random Walks with Restarts: starting node인 Q로만 teleport한다.

위의 두 개 + 기본적인 PageRank가 서로 같은 꼴로 표현된다.
$$
G = \beta M + (1-\beta)\ T
$$
여기서, $T$는 어떻게 teleport할지를 나타내는 행렬이다.

## Lecture 4.4: Matrix Factorization and Node Embeddings

$Z$를 embedding matrix(혹은 encoder)라고 하자. row의 개수는 embedding dimension이고, 각 column $Z_v$는 node $v$의 embedding vector가 된다. 만약 node $u$와 $v$가 '비슷'하다면, 이들의 내적 $Z_u^\top Z_v$는 큰 값을 가져야한다.

**Simplest Node Similarity**

Similarity를 가장 간단하게 정의하는 방법 중 하나는, 두 node가 edge로 연결되어 있으면 similar하다고 정의하는 것이다. 즉, $A$를 adjacency matrix라고 할 때, $Z^\top_u Z_v = A_{u,v}$가 되게하는 (혹은 $Z^\top Z = A$) embedding matrix $Z$를 찾는 것이다. 하지만 일반적으로 embedding dimension이 node의 개수보다 훨씬 작으므로, $A$가 $Z^\top Z$와 같은 꼴로 딱 떨어지게 factorize되지 않는다. 따라서, 다음과 같은 minimization problem을 풀어서 $Z$를 approximate하게 찾는다.
$$
\underset{Z}{\mathrm{minimize}} \left\Vert A - Z^\top Z \right\Vert_2.
$$

**Random Walk-based Similarity**

DeepWalk나 node2vec에서는 node similarity가 훨씬 복잡하게 정의되어 있는데, 이들 모두 위와 비슷하게 minimization problem을 푸는 것과 동치라는 것이 알려져있다.

예를 들어, DeepWalk는 다음과 같은 행렬을 factorize하는 것과 동치이다.
$$
\log \left( \mathrm{vol\ }(G) \left( \frac{1}{T} \sum_{r=1}^{T} \left( D^{-1} A \right)^r \right) D^{-1} \right) - \log b
$$
($\mathrm{vol\ }(G) = \sum_i \sum_j A_{i,j}$, $T$는 length of random walk, $D$는 diagonal matrix s.t. $D_{u,u} = \deg(u)$, b는 number fo negative samples.)

**Limitations**

- Cannot obtain embeddings for nodes not in the training set.
- Cannot capture structural similarity
- Cannot utilize node, edge and graph features